In [ ]:
import numpy as np
import pandas as pd
import plotnine as gg

In [ ]:
def clean_column_name(column_name):
    return (
        column_name
        .strip()
        .lower()
        .replace(' ', '_')
        .replace('-', '_')
        .replace('\n', '_')
        .replace('(', '')
        .replace(')', '')
        .replace('%', 'pecent')
        .replace('/', '_or_')
        .replace('?', '')
        .replace('__', '_')
        .replace('_dates', '_date')
    )

dict_manual_renamings = {
    'campaign_symbol': 'campaign_id',
    'activist': 'activist_name',
    'campaign_announce_date': 'campaign_announcement_date',
    'in_force_prior_to_announcement_poison_pill': 'poison_pill_in_force_prior_to_announcement',
    'adopted_in_response_to_campaign_poison_pill': 'poison_pill_adopted_in_response_to_campaign',
}

list_drop_columns = [
    
    # repeat of campaign_announce_date
    'announcement_date_date', 
    
    # unused for now
    '18_months_pre_date_stock_price',
    '1_year_pre_date_stock_price',
    '6_months_pre_date_stock_price',
    '90_days_pre_date_stock_price',
    
    '18_months_pre_date_dividends',
    '1_year_pre_date_dividends',
    '6_months_pre_date_dividends',
    '90_days_pre_date_dividends',
    
    '6_months_post_date_stock_price',
    '1_year_post_date_stock_price',
    '18_months_post_date_stock_price',
    
    '6_months_post_date_dividends',
    '1_year_post_date_dividends',
    '18_months_post_date_dividends'
    
]

list_percentage_columns = [
    '18_months_pre_date_total_return',
    '1_year_pre_date_total_return',
    '6_months_pre_date_total_return',
    '90_days_pre_date_total_return',
    'ownership_pecent_on_announcements',
    '6_months_post_date_total_return',
    '1_year_post_date_total_return',
    '18_months_post_date_total_return',
]

list_column_order = [
    
    # campaign
    'campaign_id',
    'campaign_announcement_date',
    'campaign_title',
    
    # campaign objective
    'campaign_objective_primary',
    'value_demand',
    'governance_demand',
    'activist_campaign_tactic',
    'activist_campaign_results',  # y variable
    
    # campaign objective details - board seats
    'total_number_of_board_seats',
    'number_of_board_seats_sought',
    'number_of_board_seats_gained',   # y variable
    'short_or_majority_or_full_slate',   # y variable
    
    # campaign objective details - proxy
    'proxy_proposal',
    'glass_lewis_support',
    'iss_support',
    'proxy_campaign_winner_or_result',  # y variable
    
    # activist
    'activist_id',
    'activist_name',
    'activist_group',
    
    # activist holding information
    'first_trade_date',
    'last_trade_date',
    'ownership_pecent_on_announcements',
    
    # company
    'company_id',
    'company_name',
    'sector',
    
    # comapny fundamentals
    'price_at_announcement',
    'ltm_eps_at_announcement',
    'earnings_yield_at_announcement',
    
    # company features
    'current_entity_status',
    'current_entity_detail',
    'public_before_or_after_campaign_announcement',
    'poison_pill_in_force_prior_to_announcement',
    'poison_pill_adopted_in_response_to_campaign',
    
    # dates
    '18_months_pre_announcement_date',
    '1_year_pre_announcement_date',
    '6_months_pre_announcement_date',
    '90_days_pre_announcement_date',
    '6_months_post_announcement_date',
    '1_year_post_announcement_date',
    '18_months_post_announcement_date',

    # company returns
    '18_months_pre_date_total_return',
    '1_year_pre_date_total_return',
    '6_months_pre_date_total_return',
    '90_days_pre_date_total_return',
    '6_months_post_date_total_return',  # y variable
    '1_year_post_date_total_return',  # y variable
    '18_months_post_date_total_return'  # y variable

]

# Read Raw Data

In [ ]:
df_raw = pd.read_excel('../data/FactSet_Campaign v8.xlsx', skiprows=2, na_values=['', ' ', '_', '-', 'na', 'NA', 'n.a.'])

In [ ]:
df_raw.iloc[0]

# Clean Data

In [ ]:
df_cleaning = (
    
    df_raw
    
    # rename 
    .rename(columns=clean_column_name)
    .rename(columns=dict_manual_renamings)
    
    # drop
    .drop(axis='columns', labels=list_drop_columns)
    
    # convert strings to dates based on format
    # note this fails silently for malformed dates for now
    .assign(campaign_announcement_date=lambda df: pd.to_datetime(df.campaign_announcement_date, format='%Y%m%d'))
    .assign(first_trade_date=lambda df: pd.to_datetime(df.first_trade_date, format='%Y-%m-%d %H:%M:%S'))
    .assign(last_trade_date=lambda df: pd.to_datetime(df.last_trade_date.astype(str), format='%m/%d/%Y', errors='coerce'))
    
    # extract company name and activist group from campaign title
    # note that what comes after the / can be a list of comma separated activist names, I call this activist group
    .assign(company_name=lambda df: df.campaign_title.str.split(' / ', n=1, expand=True)[0])
    .assign(activist_group=lambda df: df.campaign_title.str.split(' / ', n=1, expand=True)[1])
    
    # for categoricals, standardize to Title case
    .assign(sector=lambda df: df.sector.str.title())
    .assign(public_before_or_after_campaign_announcement=lambda df: df.public_before_or_after_campaign_announcement.str.title())
    .assign(current_entity_status=lambda df: df.current_entity_status.str.title())
    .assign(current_entity_detail=lambda df: df.current_entity_detail.str.title())
    
    # light features
    .assign(earnings_yield_at_announcement=lambda df: df.ltm_eps_at_announcement / df.price_at_announcement)
    
)

# from percentages to raw units
for percentage_column in list_percentage_columns:
    df_cleaning[percentage_column] = df_cleaning[percentage_column] / 100

# reorder
df_cleaning = (
    df_cleaning
    .loc[:, list_column_order]
    .sort_values(['campaign_id', 'campaign_announcement_date', 'campaign_title'])
)
    
df_clean = df_cleaning

In [ ]:
df_clean.iloc[0]

In [ ]:
df_clean.dtypes

In [ ]:
len(df_clean)

# Write Clean Data

In [ ]:
df_clean.to_csv('../data/clean_factset_campaign_data.csv')

# Check Data

# Campaigns

Keyed by `(campaign_id, activist_id, company_id)`.

In [ ]:
df_clean.campaign_id.nunique()

In [ ]:
df_campaign = (
    df_clean
    .groupby('campaign_id')
    .last()
    .reset_index()
)

In [ ]:
df_campaign.head(5)

In [ ]:
df_campaign.groupby('campaign_objective_primary').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign.groupby('value_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign.groupby('governance_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign[[c for c in df_campaign if 'return' in c]].describe()

# Tactics

Keyed by `(campaign_id, activist_id, company_id, activist_campaign_tactic)`.

In [ ]:
df_tactic = (
    df_clean
    .groupby('campaign_id')
    [
        'activist_id',
        'company_id',
        'activist_campaign_tactic'
    ]
    .last()
    .reset_index()
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.fillna('No or Unknown'))
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.str.split(', '))
    .explode('activist_campaign_tactic')
    .assign(activist_campaign_tactic_indicator=1)
)

In [ ]:
df_tactic.head()

In [ ]:
df_tactic.groupby('activist_campaign_tactic').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_tactics_indicators = (
    pd.pivot_table(df_tactic_indicators, index=['campaign_id'], columns=['activist_campaign_tactic'], values='activist_campaign_tactic_indicator')
    .rename(columns=clean_column_name)
    .rename(columns=lambda c: 'used_' + c + '_tactic')
)

In [ ]:
df_tactics_indicators.head(10)

# Activists

Keyed by `(activist_id)`.

In [ ]:
df_clean.activist_id.nunique()

In [ ]:
df_activist = (
    df_clean
    .groupby('activist_id')
    ['activist_name', 'activist_group']
    .last()
    .reset_index()
)

df_activist.head(5)

In [ ]:
(
    pd.merge(
        df_activist,
        df_campaign.groupby('activist_id').campaign_id.count().to_frame('campaign_count'),
        how='left',
        on=['activist_id']
    )
    .sort_values(by='campaign_count', ascending=False)
    .head(10)
)

# Targets

Keyed by `(company_id)`.

In [ ]:
df_clean.company_id.nunique()

In [ ]:
df_company = (
    df_clean
    .groupby('company_id')
    [
        'company_name',
        'sector',
        'current_entity_status',
        'current_entity_detail'
    ]
    .last()
    .reset_index()
)

In [ ]:
df_company.head(10)